In [3]:
import numpy as np
import cv2
import mediapipe as mp
# import time
from stopwatch import Stopwatch

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
# CAMERA MEDIA PIPING
state = None  # can be standing sitting lying
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imshow("Mediapipe feed: ", frame)
        # time.sleep(1)
        if True:
            # recolor image to rgb from cv2 default (bgr)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # make detection and store in result
            # writeable flag is unset before processing to improve performance and avoid unintended write ops.
            # frame is read and processed and the data is written to results.
            results = pose.process(frame)

            # convert back to cv2 default bgr
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulderStats = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            hipStats = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            ankleStats = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

            shoulder = [
                shoulderStats.x,
                shoulderStats.y,
            ]
            hip = [
                hipStats.x,
                hipStats.y,
            ]
            ankle = [
                ankleStats.x,
                ankleStats.y,
            ]
            print(shoulder[1])
            
        except:
            pass

        mp_drawing.draw_landmarks(
            image,  # output
            results.pose_landmarks,  # passing landmarks
            mp_pose.POSE_CONNECTIONS,  # passing landmark connections
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        cv2.imshow("Mediapipe feed: ", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1719925754.855629  467100 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719925754.858801  467144 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 4400 (HSW GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719925754.995276  467141 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719925755.030450  467140 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/hassaan/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype()

-0.2658044695854187
-0.25944849848747253
-0.260187566280365
-0.2820696532726288
-0.2862708270549774
-0.3547796607017517
-0.3833526074886322
-0.5184555649757385
-0.56525719165802
-0.5193840265274048
-0.48375946283340454
-0.4821338653564453
-0.4643954336643219


In [5]:
def calculateAngle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180 / np.pi)

    if angle > 180:
        return 360 - angle
    return angle

In [6]:
##for a person to be laying the difference in their head x and foot x should be small while head y and foot y is small.
# the reverse would mean that they are standing.
# this all assumes a condition where the camera is viewing the person from the side.
# what if the person has to be viewed from the front?
# the head
# z value is unreliable.

In [6]:
# CAMERA MEDIA PIPING
lying = Stopwatch(2)
sitting = Stopwatch(2)
state = None  # can be "standing", "sitting", "lying"
cap = cv2.VideoCapture(0)
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imshow("Mediapipe feed: ", frame)
        # time.sleep(1)
        if True:
            # recolor image to rgb from cv2 default (bgr)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # make detection and store in result
            results = pose.process(image)

            # convert back to cv2 default bgr
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulderStats = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            hipStats = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            ankleStats = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

            shoulder = [
                shoulderStats.x,
                shoulderStats.y,
            ]
            hip = [
                hipStats.x,
                hipStats.y,
            ]
            ankle = [
                ankleStats.x,
                ankleStats.y,
            ]

            angle = int(calculateAngle(shoulder, hip, ankle))

            cv2.putText(
                image,
                str(angle),
                tuple(np.multiply(hip, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

        except Exception as e:
            print(f"Error: {e}")
            continue

        if state != "lying" and angle > 140:
            lying.start()
            sitting.stop()
            state = "lying"

        if state == "lying" and angle < 140:
            sitting.start()
            lying.stop()
            state = "sitting"

        cv2.rectangle(image, (0, 0), (350, 50), (245, 117, 16), -1)

        cv2.putText(
            image,
            "ANGLE",
            (15, 12),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
            cv2.LINE_AA,
        )
        cv2.putText(
            image,
            str(angle),
            (15, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        cv2.putText(
            image,
            "STATE",
            (90, 12),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
            cv2.LINE_AA,
        )
        cv2.putText(
            image,
            state,
            (90, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        cv2.putText(
            image,
            "Down time",
            (150, 12),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
            cv2.LINE_AA,
        )
        cv2.putText(
            image,
            str(int(lying.duration)),
            (150, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        mp_drawing.draw_landmarks(
            image,  # output
            results.pose_landmarks,  # passing landmarks
            mp_pose.POSE_CONNECTIONS,  # passing landmark connections
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        cv2.imshow("Mediapipe feed: ", image)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1719991111.566771  472576 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719991111.574791  522920 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 4400 (HSW GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719991111.825821  522913 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719991111.862904  522915 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/hassaan/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype()